Empty Shells and Hollow Sales

Among the busy chatter of businessmen & women in suits, a stressed voice rings out above all - "It's been going on for too long, these orders just don't look right, and the numbers don't add up! There has to be something malicious going on!<span style="color: var(--vscode-foreground);">" the CEO says in the boardroom. Pointing to you, the freshly hired Database intern, ID 86, and the senior developer you're working under, she orders "You two, drop what you're doing and work on getting to the bottom of this. I'll send you the details."</span>

<span style="color: var(--vscode-foreground);">A few minutes later, after the meeting, you ask in your head, what could it be? Money laundering? Who could be doing it? An organized crime syndicate? Maybe even a foreign government? None of these are impossible, after all. But it could also just be the fault of some buggy code, especially with AI starting to become prevalent as of this year." Before you can think it through, your phone buzzes with a notification - opening it, it reads:</span>

<span style="color: var(--vscode-foreground);">From: <a href="mailto:sardavis1975@northwinds.com" data-href="mailto:sardavis1975@northwinds.com" title="mailto:sardavis1975@northwinds.com" is-markdown="true" is-absolute="false">sardavis1975@northwinds.com</a></span>

<span style="color: var(--vscode-foreground);">Hello,&nbsp;DB_INT_0086</span>

This is a message from the office of Sara Davis, CEO of NorthWinds. As you heard earlier, we've begun to suspect that some entity is using our company to move something through our company. I've decided to delegate this task to you, as all other DB teams are occupied with other necessary or urgent tasks. You're not alone in this, however, and the senior developer you're working under has provided you with some instructions, attached below.

"Hey, you might be wondering why this is a database issue and not a cybersec one. Also might be wondering why the CEO trusted an intern with such a task - Knowing her, she probably just sees potential in you. Anyways, some of the execs had a moment of genius and realized that the best way to track this is by seeing where the money and products move in the database. We're fairly sure this would most realistically be through the use of shell companies, as they're the best way to do these kinds of operations. Here's what you'll be querying for: 

Look for companies with low order quantities that also have huge order values. We don't wanna mix our data up, so stick to the last 12 months.

See if these potential shell companies all send to the same regions. 

Note any common suppliers

Note whether any employees within the company works with these shell companies and processes their transactions

Tie it together - What leads can we gain from inside the company, and better yet, if possible - Who is behind all of this? -Don Funk"

You will have until 10/26/2021 11:59 PM. I consider this to be the ultimate test - If you get me the information I need, your hiring into the company past the internship is guaranteed. Good luck and have good thinking.

In [20]:
SELECT TOP 25
    od.OrderID,
    o.CustomerID,
    SUM(od.UnitPrice * od.Quantity) AS TotalOrderValue
FROM Sales.OrderDetail AS od
JOIN Sales.[Order] AS o ON o.OrderID = od.OrderID
WHERE o.OrderDate >= DATEADD(YEAR, -4, GETDATE()) 
-- Note: No orders from 2022-25, so we simply set the story to take place in 2021 and ignore that that says -4.
GROUP BY od.OrderID, o.CustomerID
ORDER BY TotalOrderValue DESC;

(25 rows affected)

Total execution time: 00:00:00.006

OrderID,CustomerID,TotalOrderValue
10865,63,17250.00
11030,71,16321.90
10981,34,15810.00
10817,39,11490.70
10889,65,11380.00
10897,37,10835.24
11032,89,8902.50
10816,32,8891.00
10912,37,8267.40
10776,20,6984.50


<u>Database Intern 86's Notes - Part 1</u>

"Out of the top 25, a lot of the orders are collectively made up from IDs 65, 71, 39, 37 and 20, all notable for their repeat orders. However, these are not who we're considering. Shell companies usually come in large numbers with low order values each.

So the next queries will look at companyIDs **89, 76, 65, 34, 32, 24,** and **4** \- eliminate from there."

In [21]:
SELECT 
    o.CustomerID,
    COUNT(o.OrderID) AS TotalOrders
FROM Sales.[Order] AS o
WHERE o.CustomerID IN (89, 76, 65, 34, 32, 24, 4)
AND o.OrderDate >= DATEADD(YEAR, -4, GETDATE())
GROUP BY o.CustomerID
ORDER BY TotalOrders DESC;

(7 rows affected)

Total execution time: 00:00:00.010

CustomerID,TotalOrders
24,11
34,9
4,8
65,7
76,7
89,6
32,5


<u>Database Intern 86's Notes - Part 2</u>

"From here, our best leads are **32**, **89**, and maybe **76** and **65** - I'll include them just for investigative purposes. Next up, we find where they shipped their stuff to."

In [23]:
SELECT 
    o.CustomerID,
    o.ShipToCountry,
    o.ShipToRegion,
    o.ShipToCity,
    COUNT(o.OrderID) AS TotalOrders
FROM Sales.[Order] AS o
WHERE o.CustomerID IN (32, 89, 76, 65)
  AND o.OrderDate >= DATEADD(YEAR, -4, GETDATE())
GROUP BY o.CustomerID, o.ShipToCountry, o.ShipToRegion, o.ShipToCity
ORDER BY o.CustomerID, TotalOrders DESC;

(4 rows affected)

Total execution time: 00:00:00.010

CustomerID,ShipToCountry,ShipToRegion,ShipToCity,TotalOrders
32,USA,OR,Eugene,5
65,USA,NM,Albuquerque,7
76,Belgium,NULL,Charleroi,7
89,USA,WA,Seattle,6


<u>Database Intern 86's Notes - Part 3</u>

"Right there, **32** and **89** both ship to the US Northwest - Oregon and Washington States, which border each other. 

I've now got to see who handled their sales."

In [25]:
SELECT DISTINCT
    o.CustomerID,
    o.EmployeeID,
    e.EmployeeFirstName,
    e.EmployeeLastName
FROM Sales.[Order] AS o
JOIN HumanResources.Employee AS e ON e.EmployeeId = o.EmployeeID
WHERE o.CustomerID IN (32, 89)
  AND o.OrderDate >= DATEADD(YEAR, -4, GETDATE())
ORDER BY o.CustomerID, e.EmployeeLastName, e.EmployeeFirstName;

(6 rows affected)

Total execution time: 00:00:00.011

CustomerID,EmployeeID,EmployeeFirstName,EmployeeLastName
32,3,Judy,Lew
32,4,Yael,Peled
89,2,Don,Funk
89,7,Russell,King
89,3,Judy,Lew
89,4,Yael,Peled


<u>Database Intern 86's Notes - Part 4</u>

Those are the two. **Judy Lew**, and **Yael Peled.** Russel King and Don Funk don't have 32 in common, so we don't need to worry about those two. Plus, these instructions actually are pretty sound, so I don't think Funk would guide me if he was in on this.

For good measure, I'm also going to see, what suppliers their stuff came from <span style="color: var(--vscode-foreground);">for these orders.</span>

In [28]:
SELECT 
    o.CustomerID,
    o.EmployeeID,
    s.SupplierID,
    s.SupplierCompanyName AS SupplierName
FROM Sales.[Order] AS o
JOIN Sales.OrderDetail AS od 
    ON od.OrderID = o.OrderID
JOIN Production.Product AS p 
    ON p.ProductID = od.ProductID
JOIN Production.Supplier AS s 
    ON s.SupplierID = p.SupplierID
WHERE o.CustomerID IN (32, 89)
  AND o.EmployeeID IN (3, 4)
  AND o.OrderDate >= DATEADD(YEAR, -4, GETDATE())
ORDER BY o.CustomerID, o.EmployeeID, s.SupplierAddress;

(19 rows affected)

Total execution time: 00:00:00.013

CustomerID,EmployeeID,SupplierID,SupplierName
32,3,1,Supplier SWRXU
32,3,12,Supplier SVIYA
32,3,17,Supplier QZGUF
32,4,29,Supplier OGLRK
32,4,18,Supplier LVJUA
32,4,8,Supplier BWGYE
32,4,28,Supplier OAVQT
89,3,29,Supplier OGLRK
89,3,27,Supplier ZRYDZ
89,3,11,Supplier ZPYVS


<u>Database Intern 86's Notes - Part 5</u>

"I've found three supplier companies who fit the requirements. 

**OGLRK, ID 29,** supplied to both 32 and 89.

**SVIYA, ID 12,** supplied to both 32 and 89.

**BWGYE, ID 8,** supplied to both 32 and 89.

Also I've just realized I forgot to get the company names for 32 and 89, so I'll do that before I send in the data."

In [29]:
SELECT c.CustomerCompanyName,
c.CustomerId
FROM Sales.Customer AS c
WHERE c.CustomerId IN (32, 89);

(2 rows affected)

Total execution time: 00:00:00.009

CustomerCompanyName,CustomerId
Customer YSIQX,32
Customer YBQTI,89


To: [sardavis1975@northwinds.com](mailto:sardavis@gmail.com)

CC: [davfunk1987@northwinds.com](mailto:davfunk1987@northwinds.com)

Subject: NorthWinds Database Querying Results

Good evening, I hope this email finds you well.

Thanks to both of your instructions, I've managed to pinpoint two companies which seem to be moving money through our company, the suppliers they purchase from, and possibly two salespeople that have been handling their orders. I will list below the names as well as attach the ipynb file containing my queries and notes. I hope that the company may take this information and use it to further investigate and shut down the illegal operation. For the resources I have access to, I believe I have closed thecase as far as I'm able to, and I hope that this is satisfactory. Have a great night.

Suspected Shell Company Names, IDs: YSIQX, 32 & YBQTI, 89

Supplier Names, IDs: <span style="color: var(--vscode-foreground);">BWGYE, 8 &amp;&nbsp;</span>  <span style="color: var(--vscode-foreground);">SVIYA, 12 &amp;</span> OGLRK, 29

Suspected Salespeople Names, IDs: Judy Lew, 3 & Yael Peled, 4

Best,

86

\--The End--

<u>**As I've Written**</u>

<span style="color: var(--vscode-foreground);"><i style="">AI Note:</i></span>

I came up with the story, writing, etc, but I did have ChatGPT help me with formatting company emails and also with perfecting the query just below Part 4 of the notes (What suppliers were involved?).

<u>NACE</u>

Critical Thinking - To come up with a story that fit the queries, I had to develop queries that fit the story. and to do that I had to write reactively to the data. In addition, pattern recognition is the foundational skill required to solve this type of problem, which mirrors real-world analytical thinking processes more than simply query writing - I have to think ahead of time and adapt accordingly.

Technology - NorthWinds was a completely unfamiliar database, and to navigate it required skill in SQL to write and debug and even refactor queries accordingly. It was also necessary to write the story.

Communications/Teamwork & Leadership - In the story, the player character is seen to be practicing pair programming with their senior Don Funk, and in a "meta" sense, this project requires good communication and teamwork for sections of it like the video and GitHub. 

Professionalism - The documentation of the player character's progress, as well as the mock emails serve to mirror the expectations of how an actual database intern should write documentation and emails.

Career & Self-Development - Real-world problem solving requires a level of autonomy one cannot reach just by following instructions. By having Funk "push" the player in the right direction, they're allowed to discover while still making steady progress on the investigation

Equity & Inclusion - For the CEO to reach all the way down to a database intern to help solve an important company issue shows a strong sense of inclusivity. The project also implies the importance of ethical responsibility through the use of data to hold two other company members accountable.

<u>Personal Notes</u> (These can be ignored for the purpose of the assignment)

I believe I did a good job capturing the "investigation" aspect of this project. While 86 (The player character) doesn't solo close the case, it's also just mostly realistic that they can't. They're a fresh hire, and databases do provide a lot of info, but without any hard evidence from actually investigating things on-site, it's not really possible to prove anyone did something. It's also my first thing writing anything like this - anything else I've written out of pure creativity has really just not been about any form of mystery/investigative work or modern concepts like databases and companies. I just hope that above all else I made it realistic. I even went a step above to find the names of the CEO and the senior I was working under, and make them emails to write to. If there are preexisting emails, uhhh.... oops. 

I was honestly hoping for 65 to have more connections, with it being in Albuqerque, to have some more associations for purposes that may be related to a certain TV show involving _chemistry_ of sorts, but I digress. Would've been funny though.